# Import Libraries
Importing libraries are necessary for any project.
We're going to import commonly used Webscrapping and Data Analysis libraries. Make sure all libraries are installed.

In [1]:
import requests # for getting web contents
from bs4 import BeautifulSoup # for scraping web contents
import pandas as pd # for data analysis

from progressbar import ProgressBar, Percentage, Bar # for progress bar

# URL
For web scrapping it's important to have url.

## Candidate List

In [2]:
# link from where you want to scrap data
base_url = 'https://myneta.info/loksabha2019/'

# get web data
page = requests.get(base_url)
                        
# parse web data
soup = BeautifulSoup(page.content, "html.parser")

In [3]:
# find the table
# our trageted table is last

# this link have lists of links so first get all the links, all links stored in table so getting last table
table = soup.find_all('table')[-1]

# links stored in div class='items'
divs = table.find_all('div', class_="items")

links = []
cities = []

for div in divs:
    for a in div.find_all('a', href=True):
        link = a['href']
        if '/' in link:
            link = link.split('/')[-1:]
            links.append(link)
        elif '/' not in link:
            links.append(link)
        city = a.text.strip().title()
        cities.append(city)

In [4]:
# function to scrape values from each url
def url_scrraper(url,city):
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "html.parser")
    table = soup.find_all('table')[-1]
    thead = [th for th in table.find_all('th')]
    for i in thead:
        headings.append(i.text)
    headings.insert(-1,'Constituency')
    
    html_text = [tr for tr in table.find_all('tr')][1:]
    
    for text in html_text:
        td = [cells for cells in text.find_all('td')]
        if len(td)!= 0 :
            name = td[0].find('a').get_text().title()
            party = td[1].get_text()
            case = td[2].get_text()
            edu = td[3].get_text()
            age = td[4].get_text()
            ta = td[5].contents[0].strip().replace(u'Rs\xa0',u'').replace(',','')
            la = td[6].contents[0].strip().replace(u'Rs\xa0',u'').replace(',','')
            row = [name, party, case, edu, age, ta, la]
            row.insert(-1,city)

            #print(row)
            content.append(row)

In [5]:
content = []

pbar = ProgressBar(widgets=[Percentage(), Bar()], maxval=len(cities)).start()

for i in range(len(cities)):
    pbar.update(i+1)
    headings = []
    if type(links[i]) == list:
        url = base_url + links[i][0]
    else:
        url = base_url + links[i]
    url_scrraper(url, cities[i])
    
pbar.finish()

In [6]:
# save contents in a dataframe
data = pd.DataFrame(content[:], columns=headings)

## Winner List

In [7]:
# link from where you want to scrap data
winner_url = '/index.php?action=show_winners&sort=default'

# get web data
page = requests.get(base_url+winner_url)

# parse web data
soup = BeautifulSoup(page.content, "html.parser")

In [8]:
table = soup.find_all('table')[-2]
thead = [th for th in table.find_all('th')]
winner_headings = []

for i in thead:
    winner_headings.append(i.text.strip())

winner_content = []
html_text = [tr for tr in table.find_all('tr')][1:]
    
for text in html_text:
    td = [cells for cells in text.find_all('td')]
    if len(td)!= 0 :
        name = td[1].find('a').get_text()
        city = td[2].get_text().title()
        party = td[3].get_text().strip()
        case = td[4].get_text().strip()
        edu = td[5].get_text().strip()
        ta = td[6].contents[0].strip().replace(u'Rs\xa0',u'').replace(',','')
        la = td[7].contents[0].strip().replace(u'Rs\xa0',u'').replace(',','')
        row = [name, city, party, case, edu, ta, la]
        winner_content.append(row)

In [9]:
# save contents in a dataframe
winner_data = pd.DataFrame(winner_content[:], columns=winner_headings[1:])

In [10]:
winner_data = winner_data.rename(columns={'Constituency ∇':'Constituency', 'Criminal Case':'Criminal Cases'})

In [11]:
winner_data['Winner'] = 1

## Women Candidates

In [12]:
women_url = "index.php?action=summary&subAction=women_candidate&sort=candidate#summary"
page = requests.get(base_url + women_url)
soup = BeautifulSoup(page.content, 'html.parser')

women_headings = []

table = soup.find_all('table')[-1]
thead = [th for th in table.find_all('th')]

for i in thead:
    women_headings.append(i.text.strip())

women_content = []
html_text = [tr for tr in table.find_all('tr')][1:]
    
for text in html_text:
    td = [cells for cells in text.find_all('td')]
    if len(td)!= 0 :
        name = td[1].find('a').get_text()
        city = td[2].get_text().title()
        party = td[3].get_text().strip()
        case = td[4].get_text().strip()
        edu = td[5].get_text().strip()
        ta = td[6].contents[0].strip().replace(u'Rs\xa0',u'').replace(',','')
        la = td[7].contents[0].strip().replace(u'Rs\xa0',u'').replace(',','')
        row = [name, city, party, case, edu, ta, la]
        women_content.append(row)

In [13]:
# save contents in a dataframe
women_data = pd.DataFrame(women_content[:], columns=women_headings[1:])

In [14]:
women_data = women_data.rename(columns={'Candidate∇':'Candidate', 'Criminal Case':'Criminal Cases'})

In [15]:
women_data['Gender'] = 'F'

In [22]:
final_data = pd.merge(left=data, right=winner_data, 
                      how ='left', on=['Candidate', 'Criminal Cases', \
                                       'Constituency', 'Party', 'Education', \
                                       'Liabilities','Total Assets'])

final_data = pd.merge(left=final_data, right=women_data, 
                      how ='left', on=['Candidate', 'Criminal Cases', \
                                       'Constituency', 'Party', 'Education', \
                                       'Liabilities','Total Assets'])

In [23]:
final_data

,Candidate,Party,Criminal Cases,Education,Age,Total Assets,Constituency,Liabilities,Winner,Gender
0,Kuldeep Rai Sharma,INC,0,Graduate Professional,52,132233012,Andaman And Nicobar Islands,80450870,1.0,NaN
1,Ayan Mandal,AITC,0,Graduate,30,7270440,Andaman And Nicobar Islands,1500000,NaN,NaN
2,C G Saji Kumar,All India Hindustan Congress Party,0,12th Pass,48,120000,Andaman And Nicobar Islands,0,NaN,NaN
3,C U Rasheed,IND,0,12th Pass,34,202808,Andaman And Nicobar Islands,1700000,NaN,NaN
4,Gour Chandra Majumder,IND,0,Graduate,52,6062000,Andaman And Nicobar Islands,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
7963,Maksuda Khatun,CPI(M),0,Doctorate,58,12578736,Uluberia,2876353,NaN,F
7964,Minati Sarkar,SUCI(C),0,Graduate,61,1091556,Uluberia,0,NaN,F
7965,Shoma Ranisree Roy\t,INC,0,Post Graduate,40,5400000,Uluberia,0,NaN,F
7966,Simal Saren,Indian Unity Centre,0,Graduate Professional,33,946406,Uluberia,601000,NaN,NaN


In [24]:
final_data['Winner'] = final_data['Winner'].fillna(0)
final_data['Gender'] = final_data['Gender'].fillna('M')

In [29]:
final_data.Winner = final_data.Winner.astype('object')
final_data['Criminal Cases'] = final_data['Criminal Cases'].astype('int')

# Data Analysis

## Check Data Shape

In [30]:
# data shape
print(final_data.shape)

(7968, 10)


## Look at Example Records

In [31]:
# check few top rows of data
final_data.head()

,Candidate,Party,Criminal Cases,Education,Age,Total Assets,Constituency,Liabilities,Winner,Gender
0,Kuldeep Rai Sharma,INC,0,Graduate Professional,52,132233012,Andaman And Nicobar Islands,80450870,1,M
1,Ayan Mandal,AITC,0,Graduate,30,7270440,Andaman And Nicobar Islands,1500000,0,M
2,C G Saji Kumar,All India Hindustan Congress Party,0,12th Pass,48,120000,Andaman And Nicobar Islands,0,0,M
3,C U Rasheed,IND,0,12th Pass,34,202808,Andaman And Nicobar Islands,1700000,0,M
4,Gour Chandra Majumder,IND,0,Graduate,52,6062000,Andaman And Nicobar Islands,0,0,M


## Descriptive Statistics

In [32]:
# getting Generate descriptive statistics of data. Generate descriptive statistics include count, mean, std, min_value, 25%, 50%, 75%, max_value
final_data.describe()

,Criminal Cases
count,7968.000000
mean,0.573168
std,4.061399
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,240.000000


## Summary of data-type, columns, non-null values, memory usage.

In [33]:
# data info
final_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7968 entries, 0 to 7967
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Candidate       7968 non-null   object
 1   Party           7968 non-null   object
 2   Criminal Cases  7968 non-null   int64 
 3   Education       7968 non-null   object
 4   Age             7968 non-null   object
 5   Total Assets    7968 non-null   object
 6   Constituency    7968 non-null   object
 7   Liabilities     7968 non-null   object
 8   Winner          7968 non-null   object
 9   Gender          7968 non-null   object
dtypes: int64(1), object(9)
memory usage: 684.8+ KB


## Column labels

In [34]:
# get the column labels of the data.
final_data.columns

Index(['Candidate', 'Party', 'Criminal Cases', 'Education', 'Age',
       'Total Assets', 'Constituency', 'Liabilities', 'Winner', 'Gender'],
      dtype='object')

# Data Cleaning

# Save Data into CSV

In [35]:
# save data
data.to_csv('LokSabha2019.csv', index=False)